<a href="https://colab.research.google.com/github/johyunkang/adp_certificate/blob/main/adp_stats_quiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADP 27회 통계

Q1. 2년 전 제품 생산량이 100,000개, 1년 전 제품 생산량이 150,000개, 그 후 팩토리 기술의 상승으로 제품 생산량이 250,000개 되었을 때, 연평균 상승률의 대푯값을 구하시오. (반올림하여 소수점 아래 둘째자리까지 표기하시오.)

In [7]:
import numpy as np

past_2y = 100_000
past_1y = 150_000
now = 250_000

rt1 = (past_1y - past_2y)/ past_2y
print('1년전 상승률:', rt1 * 100, ' %')

rt2 = (now - past_1y) / now
print('올해 상승률:', rt2 * 100, ' %')
print('연평균 상승률:', (rt1+rt2)/2 * 100, '%')
ans = pow((250_000/100_000), (1/2)) - 1
print('연평균 상승률(CAGR):', round(ans * 100, 2))
print(round((((past_1y / past_2y) * (now / past_1y))**0.5 -1) * 100), 2, '%')
print(round(np.sqrt(rt1 * rt2) * 100, 2), '%')
print(round(((rt1 * rt2)**0.5) * 100, 2), '%')

1년전 상승률: 50.0  %
올해 상승률: 40.0  %
연평균 상승률: 45.0 %
연평균 상승률(CAGR): 58.11
58 2 %
44.72 %
44.72 %


In [2]:
print(round((pow((now/past_2y), (1/2)) - 1) * 100, 2), '%')

import statistics  as sts

n_list = [100_000, 150_000, 250_000]
rt1 = sts.mean(n_list[:2])
rt2 = sts.mean(n_list[1:])
print(rt1, rt2)
print(sts.mean(n_list))
geo_mean = sts.geometric_mean([rt1, rt2])
print(geo_mean)

print(n_list[:2], n_list[1:])


58.11 %
125000 200000
166666.66666666666
158113.88300841878
[100000, 150000] [150000, 250000]


#### NPV

In [3]:
cashflow = [-750000, 10000, 20000, 30000, 40000]
r = 0.05 # 할인률

import numpy as np
net_present_value = np.npv(r, cashflow)
print('NPV:', net_present_value)

<ipython-input-3-80a0f3b357cd>:5: DeprecationWarning: In accordance with NEP 32, the function npv was removed from NumPy version 1.20.  A replacement for this function is available in the numpy_financial library: https://pypi.org/project/numpy-financial
  net_present_value = np.npv(r, cashflow)


RuntimeError: ignored

Q2. 8건의 광고시간 측정. 평균 15.5초, 분산 3.2초 일때 90% 신뢰구간은?


In [8]:
# 정규분포를 가정하며, 집계한 광고 수가 30개 미만이므로 t검정을 사용
from scipy import stats
mu = 15.5
var = 3.2
n = 8
stats.t.interval(alpha=0.9, loc=mu, scale=np.sqrt(var/n), df=(n-1))

(14.301763280356333, 16.698236719643667)

Q3. 15개 강의 상류와 하류의 어류다양성 점수. 상류-하류는 독립이 아닌 종속
- 3-1 귀무가설과 대립가설을 설정하라

In [ ]:
# 귀무가설 : 강의 상류와 하류간 어류 다양성 점수는 상관 없다.
# 대립가설 : 강의 상류와 하류간 어류 다양성 점수는 상관이 없지 않다.

- 3-2 통계량을 구하고, 대립가설 채택 여부를 기술하라

In [13]:
# t 검정을 사용한다고 생각함
# 상류 / 하류 간 독립이 아니라고 했으므로
# 독립표본 검정(쌍검정)이 아닌 대응표본 검정을 실행해야 한다고 생각

from scipy import stats
import pandas as pd

river = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
up = [40, 38, 42, 29, 19, 27, 37, 47, 57, 56]
down = [48, 38, 40, 30, 20, 26, 35, 48, 55, 56]

df = pd.DataFrame({'river':river, 'up':up, 'down':down})
display(df.head())

# 먼저 상류와 하류 데이터에 대한 각각의 정규성 검정을 수행
# 표본의 수가 적으니 비모수 검정 해야 하나?
print('정규성검정 상류:', stats.shapiro(df.up))
print('정규성검정 하류:', stats.shapiro(df.down))
print('샘플 데이터는 p-value 가 둘 다 0.05를 초과하여 정규성을 만족하지만, 실제 시험에선 하나가 정규성을 만족하지 못 했음')

# 둘 중 하나가 정규성을 만족하지 못하여
# stats.ttest_rel을 쓰지 못하고, 윌콕슨 부호순위검정을 수행
stats.wilcoxon(df.up, df.down)

# 수행결과 p-value 가 0.05 미만으로 나와 귀무가설을 기각
# 즉, 상-하류간 어류 다양성 점수에 유의미한 차이가 있었다.

,river,up,down
0,A,40,48
1,B,38,38
2,C,42,40
3,D,29,30
4,E,19,20


정규성검정 상류: ShapiroResult(statistic=0.9646286368370056, pvalue=0.837075412273407)
정규성검정 하류: ShapiroResult(statistic=0.9580702781677246, pvalue=0.7636648416519165)


Q4. 백분위 회귀
- 4-1 주어진 데이터에 대해 백분위 50%로 잡고 백분위 회귀 모델을 구현하고 계수들에 대해 설명하라.

참조 URL : https://amaruak00.github.io/2022/12/17/adp27th.html
